In [ ]:
import numpy as np
import glob
import cv2
import os
from tensorflow.keras.applications import ResNet50
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import metrics

In [ ]:
SIZE = 224

In [ ]:
train_images = []
train_labels = []

for directory_path in glob.glob(
    "/kaggle/input/original-smote-image-processing/split/train/*"
):
    label = directory_path.split("/")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
test_images = []
test_labels = []
for directory_path in glob.glob(
    "/kaggle/input/original-smote-image-processing/split/test/*"
):
    fruit_label = directory_path.split("/")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
x_train, y_train, x_test, y_test = (
    train_images,
    train_labels_encoded,
    test_images,
    test_labels_encoded,
)

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
model = ResNet50(weights="imagenet", include_top=False, input_shape=(SIZE, SIZE, 3))

In [ ]:
for layer in model.layers:
    layer.trainable = False
model.summary()

In [ ]:
X_train_feature = model.predict(x_train)
X_train_feature = X_train_feature.reshape(X_train_feature.shape[0], -1)

X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
RF_model = RandomForestClassifier(n_estimators=50, random_state=42)
SVM_model = SVC(kernel="linear", C=1.0, random_state=42)

In [ ]:
RF_model.fit(X_train_feature, y_train)
SVM_model.fit(X_train_feature, y_train)

In [ ]:
train_prediction_RF = RF_model.predict(X_train_feature)
train_prediction_RF = le.inverse_transform(train_prediction_RF)

train_prediction_SVM = SVM_model.predict(X_train_feature)
train_prediction_SVM = le.inverse_transform(train_prediction_SVM)

In [ ]:
test_prediction_RF = RF_model.predict(X_test_features)
test_prediction_RF = le.inverse_transform(test_prediction_RF)

test_prediction_SVM = SVM_model.predict(X_test_features)
test_prediction_SVM = le.inverse_transform(test_prediction_SVM)

In [ ]:
print(
    "Random Forest Training Accuracy = ",
    metrics.accuracy_score(train_labels, train_prediction_RF),
)
print(
    "Random Forest Testing Accuracy = ",
    metrics.accuracy_score(test_labels, test_prediction_RF),
)

print(
    "SVM Training Accuracy = ",
    metrics.accuracy_score(train_labels, train_prediction_SVM),
)
print(
    "SVM Testing Accuracy = ", metrics.accuracy_score(test_labels, test_prediction_SVM)
)